In [3]:
import json
import pickle
import tqdm
import pandas as pd
from conversation import create_coder, create_reviewer, create_refiner, start_conversation

# Trabalho de RL - Qual o Melhor Prompt para Iterar sobre a Geração de Código?
Nosso trabalho de RL é sobre descobrir quais os melhores prompts para iterar sobre a geração de 
código de LLMs.

## Como funciona
Criamos uma conversa com 3 participantes: Coder, Reviewer e Refiner.
Cada participante é responsável por enviar um prompt para o LLM, e inputar os resultados do LLM no
ambiente (aqui chamado de 'conversa').

### Coder
O Coder é responsável por escrever o código.  
Ele envia o prompt inicial, descrevendo o problema a ser resolvido. Definimos que todos os nossos
problemas serão de limpeza de uma base de dados csv.  
O Coder só participa da conversa 1 vez (no início) e, por isso, não o definimos como um agente RL.
Ao invés disso, ele é programado para iterar por todos os prompts n vezes, e avaliamos os resultados
das conversas com cada prompt inicial posteriormente.

### Reviewer
O Reviewer é responsável por avaliar o código gerado pelo LLM.  
Ele envia um prompt solicitando a avaliação do código gerado pelo LLM. Ele pode essa avaliação
sempre após a geração de um código que não tem nota superior à nota terminal.  
O Reviewer é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Refiner
O Refiner é responsável por refinar o código gerado pelo LLM.
Ele envia um prompt solicitando a melhoria do código gerado pelo LLM. Ele pode essa avaliação
sempre após uma revisão do Reviewer.  
O Refiner é um agente RL, e seu objetivo é maximizar a nota do código gerado pelo LLM.

### Prompt
Para cada participante, geramos prompts que iam de 1 a $n$ nas **escalas** das seguintes **propriedades**:
- Clareza;
- Comprimento;
- Especificidade, e
- Complexidade.

Isso totalizou até 20 prompts diferentes para cada participante. Para diminuir o espaço de ações,
optamos por usar uma estratégia mais simples:

- Para cada prompt (**comprimento da escala** x **número de propriedades**) do **Coder**;
    - Para cada **propriedade** do **Reviewer**;
        - Para cada **propriedade** do **Refiner**;
            - Geramos $m$ conversas onde:
                1. O Coder envia o prompt e adiciona o código inicial;
                2. O código é avaliado (se a nota não for terminal, prossegue);
                3. O Reviewer escolhe um dos $n$ prompts da **propriedade** e adiciona a revisão;
                4. O Refiner escolhe um dos $n$ prompts da **propriedade** e adiciona a melhoria.
                5. Se o comprimento da conversa não for terminal, volta ao passo 2.

### Avaliação do Código
O código é avaliado por um LLM usando a bibliteca `instructor`. Pedimos que o código receba uma nota
de 0 a 100 para a sua corretude e legibilidade, bem como uma curta explicação do porquê da nota 
(esse comentário é adicionado posteriormente à conversa).  
Se a nota média for superior a 95, a conversa é terminada pois consideramos que o código é bom o
suficiente.

In [4]:
# List with JSON files name
json_files_coder = [
    "json_files/prompts_clarity_coder.json",
    "json_files/prompts_size_coder.json",
    "json_files/prompts_specificity_coder.json",
    "json_files/prompts_complexity_coder.json"
]

json_files_reviewer = [
    "json_files/prompts_clarity_reviewer.json",
    "json_files/prompts_size_reviewer.json",
    "json_files/prompts_specificity_reviewer.json",
    "json_files/prompts_complexity_reviewer.json"
]

json_files_refiner = [
    "json_files/prompts_prop1_refiner.json",
    "json_files/prompts_prop2_refiner.json",
    "json_files/prompts_prop3_refiner.json",
    "json_files/prompts_prop4_refiner.json"
]

prompts_coder = []
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for i, item in enumerate(data):
            item["index"] = i
        prompts_coder += data

reviewer_properties = {}    
for file_name in json_files_reviewer:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in reviewer_properties:
                reviewer_properties[item["propriedade"]] = []
            reviewer_properties[item["propriedade"]].append(item['prompt'])

refiner_properties = {}
for file_name in json_files_coder:
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)
        for item in data:
            if item["propriedade"] not in refiner_properties:
                refiner_properties[item["propriedade"]] = []
            refiner_properties[item["propriedade"]].append(item['prompt'])                      

In [5]:
'''
MAX_TURNS = 1
TOT_CONVERSATIONS = 2
coder = create_coder(prompts_coder)

for coder_prompt_dict in prompts_coder:
    for rev_prop, rev_prompts in reviewer_properties.items():
        for ref_prop, ref_prompts in refiner_properties.items():
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                try:
                    start_conversation(
                        coder, 
                        coder_prompt_dict, 
                        reviewer, 
                        refiner, 
                        MAX_TURNS
                    )
                except Exception as e:
                    print(f"Conversation Skipped: {e}")
            # Saving the models
            with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(reviewer, file)
            with open(f"models/refiner_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:
                pickle.dump(refiner, file)
'''                

'\nMAX_TURNS = 1\nTOT_CONVERSATIONS = 2\ncoder = create_coder(prompts_coder)\n\nfor coder_prompt_dict in prompts_coder:\n    for rev_prop, rev_prompts in reviewer_properties.items():\n        for ref_prop, ref_prompts in refiner_properties.items():\n            reviewer = create_reviewer(rev_prompts)\n            refiner = create_refiner(ref_prompts)\n            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):\n                try:\n                    start_conversation(\n                        coder, \n                        coder_prompt_dict, \n                        reviewer, \n                        refiner, \n                        MAX_TURNS\n                    )\n                except Exception as e:\n                    print(f"Conversation Skipped: {e}")\n            # Saving the models\n            with open(f"models/reviewer_{coder_prompt_dict}_{rev_prop}_{ref_prop}.pkl", "wb") as file:\n                pickle.dump(reviewer, file)\n    

In [7]:
from IPython.display import display, Markdown
from rl.code_evaluator import CodeEvaluator


MAX_TURNS = 1
TOT_CONVERSATIONS = 2
coder = create_coder(prompts_coder)

for i, coder_prompt_dict in enumerate(prompts_coder):
    for j, (rev_prop, rev_prompts) in enumerate(reviewer_properties.items()):
        for k, (ref_prop, ref_prompts) in enumerate(refiner_properties.items()):
            reviewer = create_reviewer(rev_prompts)
            refiner = create_refiner(ref_prompts)
            evaluator = CodeEvaluator(environment=None, prompt="Evaluate the code quality", name="Code Evaluator")
            
            for _ in tqdm.tqdm(range(TOT_CONVERSATIONS), desc="Conversations", position=0):
                #try:
                    environment = start_conversation(
                        coder, 
                        coder_prompt_dict, 
                        reviewer, 
                        refiner, 
                        MAX_TURNS
                    )
                    for message in environment.messages:
                        display(Markdown(f"**{message['role']}**: {message['content']}"))
                
                #except Exception as e:
                #    print(f"Conversation Skipped: {e}")

            # Salva os modelos
            with open(f"models/reviewer_{i}_{j}_{k}.pkl", "wb") as file:
                pickle.dump(reviewer, file)
            with open(f"models/refiner_{i}_{j}_{k}.pkl", "wb") as file:
                pickle.dump(refiner, file)
                              


Conversations:   0%|          | 0/2 [00:00<?, ?it/s]

**User**: I gotta clean up this messed-up CSV file...  Python, please! 🥺

Here is the `imdb_sample_10.csv` file content:

Series_Title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,Director,No_of_Votes
The Shawshank Redemption,1994,,Drama,nan,nan,,nan
,1972,,"Crime, Drama",nan,nan,,nan
The Dark Knight,,,"Action, Crime, Drama",,,Christopher Nolan,
The Godfather: Part II,nan,,"Crime, Drama",9.0,90.0,Francis Ford Coppola,nan
,1957,,"Crime, Drama",9.0,96.0,Sidney Lumet,nan
The Lord of the Rings: The Return of the King,2003,201 min,,8.9,94.0,Peter Jackson,nan
,,154 min,"Crime, Drama",,,,
Schindler's List,,195 min,,,,,
Inception,,148 min,"Action, Adventure, Sci-Fi",,,Christopher Nolan,
Fight Club,nan,139 min,Drama,8.8,66.0,David Fincher,nan


**assistant**: Sent by Coder: 

You got it! Let's clean up that CSV file using Python. Here's a breakdown of how to tackle this and some key code snippets:

**1. Importing Libraries**
   ```python
   import pandas as pd 
   ```
   We use the `pandas` library for its powerful data manipulation capabilities.  

**2. Loading the CSV**
   ```python
   data = pd.read_csv('imdb_sample_10.csv')
   print(data.head()) # Look at the first few rows to inspect 
   ```

**3. Cleaning and Formatting**

* **Handling Missing Values (NaNs):**  We want to replace these with more sensible values, like empty strings.
    ```python
    data['IMDB_Rating'] = data['IMDB_Rating'].fillna('')  
    data['Meta_score'] = data['Meta_score'].fillna('') 

    print(data) # See the updated DataFrame
    ```
* **Dealing with Unnecessary Data**  We'll drop the unwanted column like "No_of_Votes".
    ```python 
    data = data.drop(['No_of_Votes'], axis=1) 
   ```
    
* **Formatting and Cleaning Titles:**  Convert title entries into something more standardized
 ```python
    data['Series_Title'] = data['Series_Title'].str.strip()
     
    print(data.head())
    ```

**4. Saving the Cleaned Data (Optional)**
   ```python
   data.to_csv('cleaned_imdb_sample.csv', index=False) 
   ```



**Explanation:**

* **`pd.read_csv()`:**  This reads your CSV file and creates a pandas DataFrame – a structured table-like data object.
* **`.head()`:** Displays the first few rows of the DataFrame to get an idea of what you're working with. 
* **`fillna('')`:**  Replaces `NaN` values (Not a Number) in columns like "IMDB_Rating" and "Meta_score" with empty strings, which is often better than leaving them as `NaN`.

**Important Notes**


* **Error Handling:** If you encounter unexpected errors or missing data, it's best to include error handling.
* **Data Validation:** Before applying any transformations or cleaning, it's good practice to check your data for consistency and make sure it meets your needs. 



Let me know if you have more specific requirements about how you want to clean your dataset!  

**assistant**: Sent by Code Evaluator: 

**Correctness:** 90

**Grade Explanation:** The code leverages the pandas library for its efficient data manipulation capabilities and uses clear logic to handle missing values, remove irrelevant columns and format titles. This demonstrates a good understanding of cleaning data and setting up data structures.

**Readability:** 85

**Grade Explanation:** The explanation is generally well-structured with step-by-step explanations for each code section but can benefit from more detailed context around the specific libraries being used when referring to them.

**assistant**: Sent by Reviewer: 

You got it! Let's take a deeper dive into that Python program and see if we can sharpen its performance. 

**Reviewing the Code:**

The code you provided shows an attempt to clean up a CSV file, but there are some issues:


* **Inconsistent Formatting:**  It seems like data is being cleaned in different ways (like converting titles).
* **Missing Context:** The code jumps straight into actions without providing clear instructions about what it's trying to do or where the data is coming from. 

**Here’s an improved approach, focusing on clarity and better organization**:


```python
import pandas as pd

# Load the CSV file into a Pandas DataFrame
data = pd.read_csv('imdb_sample_10.csv')  # Replace with your file path

print(data.head()) # Look at first few rows

# 1. Data Cleaning: Addressing Missing Values & Column Formatting

# Replacing NaN values with empty strings (good practice)
data['IMDB_Rating'] = data['IMDB_Rating'].fillna('')
data['Meta_score'] = data['Meta_score'].fillna('') 
print(data)  

# 2. Data Manipulation: Removing Unnecessary Columns 
data = data.drop(['No_of_Votes'], axis=1) # Removing unnecessary column 

# 3. Formatting:  Clean Titles and Other Considerations 
# ... (Add specific formatting here based on your requirements)


# Save the cleaned DataFrame to a new CSV file
data.to_csv('cleaned_imdb_sample.csv', index=False)  
```



**Explanation:**

* **Clarity First:** The code is now structured and explained. 
    * `print(data.head())` shows us the first few rows, which helps you inspect your data before and after cleaning. 


* **Comments:** Adding comments to explain what's going on will make it easier for others (and for you) to understand.

**Tips for Reviewers**
   
  You can always ask me:
    * **"What does this code do?"** 
    * **"How is the code using Pandas?"**
    * **"Could you explain the logic behind certain steps?"** 


I'm here to help you polish your code and make it even more robust. Let's work together!



Let me know what parts you want to tackle next. I can help with formatting, addressing edge cases, or suggesting additional improvements!  


**assistant**: Sent by Code Refiner: 

You got it! Let's tackle that messy CSV file using Python. I'll guide you through the process and clear things up along the way! Here's a breakdown of how we can do this. 

**1. Loading Your Data (Pandas to the Rescue)**
   ```python
   import pandas as pd 

   data = pd.read_csv('your_file.csv') # Replace 'your_file.csv' with your actual file path!
   print(data.head())  # Quick look at first few rows
   ```

**Explanation:**
* **`import pandas as pd`**: We bring in the powerful pandas library, which is our tool for working with data. The `pd` short form makes things easier to use. 
* **`data = pd.read_csv('your_file.csv')`**:  This tells pandas where your CSV file is located and reads it into a DataFrame. This is our structured table! 

**2. Cleaning Up Your Data (The Power of Pandas)**
   ```python
   # Dealing with missing data: 
   data['Column_Name'].fillna(0) # Replace 'Column_Name' with the actual column name
   print(data.head())  
   ```
    
**Explanation:**

* **`data['Column_Name'] = data['Column_Name'].fillna(0)`**: We fill `NaN` (Not a Number) values in specific columns like 'MovieTitle' with 0 to make it easier! You'll likely need to choose the right column name based on your data. 
   

**3. Formatting and Cleaning Up:**
   ```python
   data['Column_Name'] = data['Column_Name'].str.title() # Make title case 
   print(data.head())  
   ``` 
    
   **Explanation:**
* **`data['Column_Name'] = ...`**: We use `str.title()` to capitalize the first letter of each word in 'Column_name' (and 'MovieTitle') to make them easier to read!

**4. Saving Your Cleaned Data:**
   ```python
   data.to_csv('cleaned_data.csv', index=False)  # Make sure you replace 'cleaned_data.csv' with your desired file name 
   ``` 


**Explanation:**

* **`data.to_csv('cleaned_data.csv', index=False)`**: Saves our clean DataFrame as a new CSV file! 


**Important Notes:**

* **File Path:** Make sure you replace `"your_file.csv"` with the correct path to your actual CSV file on your computer.
* **Debugging:** If you run into any issues, try running this code and see what happens! We can use Python's error-handling capabilities if needed.  


Let me know what questions you have or what parts you want help with specifically! 


Conversations:  50%|█████     | 1/2 [01:18<01:18, 78.32s/it]


InstructorRetryException: 4 validation errors for CodeEvaluation
correctness_grade
  Field required [type=missing, input_value={'properties': {'correctn...planation_readability']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
explanation_correctness
  Field required [type=missing, input_value={'properties': {'correctn...planation_readability']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
readability_grade
  Field required [type=missing, input_value={'properties': {'correctn...planation_readability']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
explanation_readability
  Field required [type=missing, input_value={'properties': {'correctn...planation_readability']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing